In [55]:
import numpy as np

In [163]:
%run data_preprocessing/credit_card_data.ipynb

In [164]:
%run logistic_regression.ipynb

In [176]:
data = np.array(credit_card_data)
data.shape

(30000, 24)

In [169]:
def get_new_model():
    return LogisticRegression()

In [170]:
def evaluate_acc(y,yh):
    tp = np.sum(np.logical_and(yh==1,y==1))
    tn = np.sum(np.logical_and(yh==0,y==0))
    
    acc = (tp+tn)/y.size
    return acc

In [171]:
def get_folds(k, data):
    np.random.shuffle(data)
    
    # Split into k-folds and remove empty splits if not appropriate size
    folds = np.array_split(data, k)
    folds = [x for x in folds if x.size > 0]
    
    return folds

In [191]:
def k_fold_cross_validation(k, data):
    folds = get_folds(k,data)
    fold_accuracies = []
    for i in range(k):
        model = get_new_model()
        
        # Get test row, and training set
        test = folds[i]
        train = np.delete(folds, i, 0)
        print(train)
        
        train_X = train[:, :-1]
        train_Y = train[:, -1]
        
        # Fit and predict
        model.fit(train_X, train_Y)
        test_X = test[:, :-1]
        test_Y = test[:, -1]
        model_results = model.predict(test_X)
        
        fold_accuracies.append(evaluate_acc(model_results,test_Y))
    return np.average(fold_accuracies)

In [192]:
k_fold_cross_validation(2000,data)

[[[ 30000      2      2 ...    317      0      1]
  [ 70000      2      2 ...   1000      0      1]
  [ 50000      1      2 ...    514    335      0]
  ...
  [310000      2      3 ...    500    500      0]
  [200000      1      1 ...   1002   1002      0]
  [220000      1      1 ...      0      0      0]]

 [[ 30000      2      2 ...   1000    513      0]
  [ 50000      2      2 ...   1159   1050      0]
  [ 50000      2      2 ...   1500   1000      0]
  ...
  [520000      2      3 ...  14000  15200      0]
  [ 30000      2      5 ...    486      0      0]
  [ 90000      2      3 ...   1500   1500      1]]

 [[ 50000      1      3 ...   7881   1000      0]
  [200000      2      2 ...   5519   2358      0]
  [ 80000      1      1 ...   2500      0      0]
  ...
  [130000      2      1 ...      0   1313      0]
  [ 30000      2      1 ...   2400   1100      1]
  [ 80000      2      1 ...  59349      0      0]]

 ...

 [[180000      1      2 ...      0      0      0]
  [ 30000      1    

ValueError: too many values to unpack (expected 2)